In [129]:
import pandas as pd,numpy as np

In [130]:
import mysql.connector as connection

In [131]:
mydb = connection.connect(host="sendapp-prod-read-replica.cvjc76vwubit.ap-southeast-2.rds.amazonaws.com", 
                          database = 'sendApp',user="ujjwal", passwd="UjJWl648759",use_pure=True)


query ="with T1 as \
( \
select uo.orderId as order_id,date(uo.createdAt) as order_date,uo.noRush,ua.postalCode,uo.id as user_id, \
concat(u.firstName,'_',u.lastName) as customer_name,sp.firstName as rider_name,z.name as store_name, \
round(time_to_sec(substring(timediff(substring(uo.handoff,12,8),time(uo.createdAt)),1,8))/60,2) as pickup_time, \
round(time_to_sec(substring(timediff(substring(uo.arrived,12,8),time(uo.createdAt)),1,8))/60,2) as total_del_time \
from \
userorder as uo \
left join useraddress as ua \
on uo.addressId = ua.id \
left join zone as z \
on uo.zoneId = z.id \
left join shopper as sp \
on uo.riderId = sp.id \
left join user as u \
on uo.userId = u.id \
where date(uo.createdAt) >= DATE(NOW()) + INTERVAL -7 DAY \
and time_to_sec(substring(timediff(substring(uo.handoff,12,8),time(uo.createdAt)),1,8))/60  > 0 \
and time_to_sec(substring(timediff(substring(uo.arrived,12,8),time(uo.createdAt)),1,8))/60  > 0),T2 as ( \
WITH t1 as (select b.orderId, b.userId, b.zoneId, b.price, b.promoAmount, b.promoId, b.order_date, b.create_time,b.hours, \
str_to_date(SUBSTRING(arriv,12,8),'%H:%i:%s') AS arrived_time from (SELECT orderId, userId, zoneId, price, promoAmount, \
promoId,DATE(createdAt) as order_date,time(createdAt)as create_time,HOUR(createdAt) as hours,nullif(arrived,'N/A') as arriv \
FROM userorder Where date(createdAt) >= DATE(NOW()) + INTERVAL -7 DAY ) b), \
t2 as (select prod_id, sale_price,order_ID, user_ID,SUM(quantity) as tot_qty, SUM(g_m_v) as gmv, store,zone_ID \
from (Select x.product_id as prod_id, x.sp as sale_price,x.order_ID, x.user_id as user_ID,x.qty as quantity, \
(x.qty * x.sp) as g_m_v, x.STORE_L as store,x.zone_id as zone_ID \
from (select uop.productId as product_id, uop.orderId as order_id, uop.userId as user_id,uop.salePrice as sp, \
uop.quantity as qty, uop.zoneId as zone_id, z.name as STORE_L from userorderedproduct as uop left join zone as z \
on z.id = uop.zoneId) as x) as b group by order_ID,user_ID,zone_ID,store) \
select t1.orderId, t1.userId, t1.order_date,t2.gmv as GMV from t1 left join t2  on t1.orderId = t2.order_ID and \
t1.userId = t2.user_ID \
group by 1) \
select T1.order_id,round(T2.GMV,2) as GMV_$,T1.order_date,T1.noRush,T1.postalCode,T1.user_id, \
T1.customer_name,T1.rider_name,T1.store_name,T1.pickup_time as pickup_time,T1.total_del_time as total_del_time \
from T1 inner join T2 \
on T1.order_id = T2.orderId \
group by 1 \
order by 3 desc;"


result_dataFrame = pd.read_sql(query,mydb)

mydb.close()

In [132]:
result_dataFrame.shape

(1348, 11)

In [133]:
result_dataFrame.tail()

,order_id,GMV_$,order_date,noRush,postalCode,user_id,customer_name,rider_name,store_name,pickup_time,total_del_time
1343,1650959947455,15.95,2022-04-26,0,2022,f818243a-6c6f-4213-852c-dc1fe439b534,Jack_Robbins,Rico,POTTS POINT,1.53,18.98
1344,1650962115470,39.80,2022-04-26,0,3002,f9f52ac7-4b0a-40ca-a31e-2fa9894c8f5c,Vishan_Jayasinghe,caleb,Collingwood,2.50,10.60
1345,1650948219702,2.30,2022-04-26,0,2010,fa5efb3d-9d07-4c03-9063-90d19fe9f5fb,Alex_Andrews,Yida,POTTS POINT,1.77,7.70
1346,1650961944626,50.33,2022-04-26,1,2096,fcdeeae5-610a-4662-ba12-7a4d3250526b,Jordan_Lonergan,James Allan,Manly,3.58,9.38
1347,1650959702209,32.12,2022-04-26,0,2010,fe1b8a91-a8cd-4eac-b811-69ae6e09e49d,Gary_Ansell,Adam,POTTS POINT,1.65,7.23


In [134]:
result_dataFrame.order_date.value_counts()

2022-04-26    251
2022-04-27    228
2022-04-28    218
2022-04-30    204
2022-05-01    193
2022-04-29    178
2022-05-02     76
Name: order_date, dtype: int64

In [135]:
result_dataFrame['noRush'] = result_dataFrame['noRush'].replace([1,0],['NoRush_Order','Normal_Order'])

In [136]:
df_pivot= np.round(result_dataFrame.pivot_table(['GMV_$','pickup_time','total_del_time'],
                             index = ['store_name','noRush','postalCode'],aggfunc=np.mean),2)
df_pivot.head(60)

GMV_$  pickup_time  total_del_time
store_name  noRush       postalCode                                    
ALEXANDRIA  NoRush_Order 2008        21.58         1.69           14.89
                         2015        23.41         1.57            6.53
                         2016        33.91         3.99           15.84
                         2017        41.80         2.99           13.37
                         2018        30.53         1.83           10.55
                         2033        42.32         2.18           26.22
                         2042        40.47         2.63           14.88
                         2043        20.57         2.90           13.22
                         2044        77.42         1.78           14.28
            Normal_Order 2008        20.82         1.42           13.31
                         2015        24.43         3.22           10.12
                         2016        21.79         2.71           14.32
                         2017        26.45         2.82           12.53
                         2018        19.22         1.42           11.85
                         2033        27.62         4.08           20.34
                         2042        28.75         1.86           14.64
                         2043        23.44         2.82           13.29
                         2044         8.72         1.67           10.67
ANNANDALE   NoRush_Order 2037        25.44         2.79            7.91
                         2038        23.57         1.82            6.90
                         2039        55.40         1.83           23.17
                         2040        15.16         1.17            5.37
                         2042        17.89         1.81            8.33
                         2048        17.64         2.17            9.18
                         2049        56.12         1.76            8.68
                         2050        23.49         1.55            7.44
            Normal_Order 2037        30.79         1.71            7.92
                         2038        25.42         2.07            6.71
                         2039        65.38         4.46           20.38
                         2040        17.60         1.83            9.96
                         2042        20.89         1.74            8.68
                         2048        20.95         1.49            7.64
                         2049        50.26         2.50           14.55
                         2050        30.36         2.06            8.61
                         2204        30.45         1.75           15.50
Balaclava   NoRush_Order 3181        33.67         7.49           18.31
                         3182        39.94         3.82           10.22
                         3183        21.66         5.58            9.15
                         3184        30.40        12.13           19.24
                         3185        18.70         2.57            9.79
            Normal_Order 3181        31.41         4.76           17.64
                         3182        20.55        11.96           18.90
                         3183        23.32         2.25            7.16
                         3184        25.82         4.98           15.64
                         3185        28.88         1.86            9.26
Collingwood NoRush_Order 3002        31.71         1.03            8.17
                         3065        39.95         3.01            9.52
                         3066        23.01         1.82            5.35
                         3067        25.88         1.46            9.49
                         3068        24.20         1.98           16.99
                         3121        39.87         2.75           10.16
            Normal_Order 3002        29.41         2.52            9.50
                         3053        33.72         2.17            9.67
                         3065        31.12         1.74            8.86
     

In [137]:
data_list =[result_dataFrame,df_pivot]

In [138]:
from pandas import ExcelWriter
import base64
from io import BytesIO
fp = BytesIO()

writer = ExcelWriter(fp)

In [139]:
data_list ={'data':result_dataFrame,'summary':df_pivot}

In [140]:
for key,value in data_list.items():
   value.to_excel(writer,key)

In [141]:
writer.save()

In [142]:
import smtplib
import mimetypes
from email.message import EmailMessage
message = EmailMessage()
sender = "ujjwal@sendapp.com.au"
recipient = "robr@sendapp.com.au"
message['From'] = "ujjwal@sendapp.com.au"
message['To'] = recipient
message['CC'] = "danish@sendapp.com.au"
message['Subject'] = 'Reporting for ride times for the past week'
body = """Hello Rob
Attached report mentioning average pickup time ,average total delivery time and average GMV(Revenue) for all orders vs postcodes for the past week.
Regards"""
message.set_content(body)
mime_type, _ = mimetypes.guess_type('robbs_report.xlsx')
mime_type, mime_subtype = mime_type.split('/')
message.add_attachment(fp.getvalue(),
maintype=mime_type,
subtype=mime_subtype,
filename='robbs_report.xlsx')
print(message)
mail_server = smtplib.SMTP_SSL('smtp.gmail.com')
mail_server.set_debuglevel(1)
mail_server.login("ujjwal@sendapp.com.au", 'jeqclpnhqbyeokaw')
mail_server.send_message(message)
mail_server.quit()


From: ujjwal@sendapp.com.au
To: robr@sendapp.com.au
CC: danish@sendapp.com.au
Subject: Reporting for ride times for the past week
MIME-Version: 1.0
Content-Type: multipart/mixed; boundary="===============4671839653131127426=="

--===============4671839653131127426==
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: quoted-printable

Hello Rob
Attached report mentioning average pickup time ,average total delivery time a=
nd average GMV(Revenue) for all orders vs postcodes for the past week.
Regards

--===============4671839653131127426==
Content-Type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="robbs_report.xlsx"
MIME-Version: 1.0

...

--===============4671839653131127426==--



send: 'ehlo [192.168.1.246]\r\n'
reply: b'250-smtp.gmail.com at your service, [122.161.75.216]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [122.161.75.216]\nSIZE 35882577\n8BITMIME\nAUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'AUTH PLAIN AHVqandhbEBzZW5kYXBwLmNvbS5hdQBqZXFjbHBuaHFieWVva2F3\r\n'
reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<ujjwal@sendapp.com.au> size=198931\r\n'
reply: b'250 2.1.0 OK e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp'
send: 'rcpt TO:<robr@sen

reply: b'250 2.0.0 OK  1651469611 e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1651469611 e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp'
data: (250, b'2.0.0 OK  1651469611 e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp'


(221,
 b'2.0.0 closing connection e8-20020a170902744800b0015e8d4eb1fcsm3721851plt.70 - gsmtp')